# Imports

In [168]:
import pandas as pd

# Load

In [222]:
def load_dataframe_from_url(url, encoding = 'utf-8'):
    path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
    df = pd.read_csv(path, encoding = encoding)
    return df


url = "https://drive.google.com/file/d/1F3L_igH1kJ_Rx44rKRjRVAtrPMulYRsY/view?usp=sharing"
df = load_dataframe_from_url(url, encoding = 'ISO-8859-1')

# Punto 1: crear datetime index 

In [170]:
print('Hay %g lineas sin año' % (df.Year.isnull().sum()))
df = df[df.Year.isnull() == False].copy() #elimino filas sin año

Hay 117 lineas sin año


In [171]:
dateindex = pd.DatetimeIndex(pd.to_datetime(df.Year,  format='%Y')).copy()
serie_atletas =  df.set_index(dateindex).Athlete
serie_atletas

Year
1976-01-01             KÖHLER, Christa
1976-01-01         KOSENKOV, Aleksandr
1976-01-01        BOGGS, Philip George
1976-01-01    CAGNOTTO, Giorgio Franco
1976-01-01      WILSON, Deborah Keplar
                        ...           
2008-01-01          GUENOT, Christophe
2008-01-01         KVIRKELIA, Manuchar
2008-01-01            RAHIMOV, Vitaliy
2008-01-01              GUENOT, Steeve
2008-01-01               LOPEZ, Mijain
Name: Athlete, Length: 15316, dtype: object

# Punto 2: Separar el nombre en dos columnas

In [172]:
df[['Surname', 'Name']] = df['Athlete'].apply(lambda x: pd.Series(str(x).split(",", maxsplit=1)))
df

,City,Year,Sport,Discipline,Event,Athlete,Gender,Country_Code,Country,Event_gender,Medal,Surname,Name
0,Montreal,1976.0,Aquatics,Diving,3m springboard,"KÖHLER, Christa",Women,GDR,East Germany,W,Silver,KÖHLER,Christa
1,Montreal,1976.0,Aquatics,Diving,3m springboard,"KOSENKOV, Aleksandr",Men,URS,Soviet Union,M,Bronze,KOSENKOV,Aleksandr
2,Montreal,1976.0,Aquatics,Diving,3m springboard,"BOGGS, Philip George",Men,USA,United States,M,Gold,BOGGS,Philip George
3,Montreal,1976.0,Aquatics,Diving,3m springboard,"CAGNOTTO, Giorgio Franco",Men,ITA,Italy,M,Silver,CAGNOTTO,Giorgio Franco
4,Montreal,1976.0,Aquatics,Diving,10m platform,"WILSON, Deborah Keplar",Women,USA,United States,W,Bronze,WILSON,Deborah Keplar
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15428,Beijing,2008.0,Wrestling,Wrestling Gre-R,66 - 74kg,"GUENOT, Christophe",Men,FRA,France,M,Bronze,GUENOT,Christophe
15429,Beijing,2008.0,Wrestling,Wrestling Gre-R,66 - 74kg,"KVIRKELIA, Manuchar",Men,GEO,Georgia,M,Gold,KVIRKELIA,Manuchar
15430,Beijing,2008.0,Wrestling,Wrestling Gre-R,55 - 60kg,"RAHIMOV, Vitaliy",Men,AZE,Azerbaijan,M,Silver,RAHIMOV,Vitaliy
15431,Beijing,2008.0,Wrestling,Wrestling Gre-R,60 - 66kg,"GUENOT, Steeve",Men,FRA,France,M,Gold,GUENOT,Steeve


# Punto 3: Cantidad de medallas

In [208]:
medallas = df.pivot_table(index='Country', columns='Medal', values='City', aggfunc='count').fillna(0)
medallas['Total'] = medallas.sum(axis=1)
medallas['prop'] = medallas.Gold / medallas.Total
masganador = medallas.sort_values('Total', ascending=False).reset_index().loc[0,'Country']
masoro = medallas.sort_values('Gold', ascending=False).reset_index().loc[0,'Country']
mejorprop = medallas[medallas.Total > 1].sort_values('prop', ascending=False).reset_index().loc[0,'Country']

print('El país que mas medallas gano en total es ' + masganador)
print('El pais que mas medalles de oro gano es ' + masoro)
print('El pais con mayor proporcion de medallas de oro es ' + mejorprop)


El país que mas medallas gano en total es United States
El pais que mas medalles de oro gano es United States
El pais con mayor proporcion de medallas de oro es Cameroon


# Punto 4: Cantidad de medallas obtenidas por año

In [227]:
medallas_m = df[df.Gender == 'Men'].pivot_table(index= 'Year', columns=['Medal'], values = 'City', aggfunc='count',margins=True,margins_name='Total')
medallas_m

Medal,Bronze,Gold,Silver,Total
Year,,,,
1976.0,321,302,301,924
1980.0,328,317,313,958
1984.0,338,320,315,973
1988.0,352,334,325,1011
1992.0,391,358,356,1105
1996.0,374,356,352,1082
2000.0,386,370,370,1126
2004.0,376,361,362,1099
2008.0,391,362,357,1110
